In [1]:
import os
import os.path as osp
from glob import glob
import re
from pprint import pprint
from tqdm import tqdm
import pydicom as dicom
import random
import csv

PATHS = [f'/datasets/mimic/cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p{s}' for s in range(10, 20)]
TEXT_PATH_ROOT = '/datasets/mimic/cxr/physionet.org/files/mimic-cxr/2.0.0/files/'

get_report_path = lambda img_path: Path.joinpath(TEXT_ROOT, img_path.parents[0]).with_suffix(".txt")

def check_jpg_extension(filename):
    pattern = r"\.jpg$"
    if re.search(pattern, filename):
        return True
    else:
        return False

In [2]:
import csv

splits = {
    "train": set(),
    "validate": set(),
    "test": set(),
}
with open("/datasets/mimic/cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-split.csv", "r") as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i == 0:
            continue
        splits[row[-1]].add(f"p{row[-2]}")

In [3]:
[len(splits[x]) for x in list(splits.keys())]

[64586, 500, 293]

In [4]:
ALL_IMG_PATHS = {
    "train": list(),
    "validate": list(),
    "test": list(),
}

ALL_TXT_PATHS = {
    "train": list(),
    "validate": list(),
    "test": list(),
}

In [5]:
get_txt_path = lambda s: TEXT_PATH_ROOT + "/".join(s.split("/")[10:-1]).split(".")[0] + ".txt"
get_new_img_path = lambda s: "/".join(s.split("/")[10:])

In [6]:
for PATH in PATHS:
    top_dirs = [dir for dir in os.listdir(PATH) if dir.find(".") == -1]
    for dir_name in tqdm(top_dirs):
        paths = os.listdir(osp.join(PATH, dir_name))
        img_dirs = [osp.join(PATH, dir_name, dir) for dir in paths if dir.find(".") == -1]
        for img_dir in img_dirs:
            img_paths = [osp.join(img_dir, img_name) for img_name in os.listdir(img_dir)]
            img_paths = [path for path in img_paths if check_jpg_extension(path)]
            for sname, split_set in splits.items():
                if dir_name in split_set:
                    ALL_IMG_PATHS[sname].extend([get_new_img_path(s) for s in img_paths])
                    ALL_TXT_PATHS[sname].extend([get_txt_path(s) for s in img_paths])
                    break
            else:
                assert False, "No split is defined"
                

100%|██████████| 6579/6579 [00:54<00:00, 120.52it/s]


In [7]:
for sname, split in ALL_TXT_PATHS.items():
    for p_idx in range(len(split)):
        path = split[p_idx]
        subdirs = path.split("/")
        new_subdir = subdirs[9][:3]
        subdirs.insert(9, new_subdir)
        path = "/".join(subdirs)
        split[p_idx] = path

In [8]:
ALL_TXT_PATHS_PATIENT = {"train": {}, "validate": {}, "test": {}}

In [10]:
for sname, split in ALL_TXT_PATHS.items():
    for p_idx in range(len(split)):
        path = split[p_idx]
        subdirs = path.split("/")
        patient_id = subdirs[10]
        study_id = subdirs[11]
        if patient_id not in ALL_TXT_PATHS_PATIENT[sname]:
            ALL_TXT_PATHS_PATIENT[sname][patient_id] = {study_id: path}
        elif study_id not in ALL_TXT_PATHS_PATIENT[sname][patient_id]:
            ALL_TXT_PATHS_PATIENT[sname][patient_id][study_id] = path

In [11]:
idx = random.randint(0, len(ALL_TXT_PATHS)-1)
f = open("/datasets/mimic/cxr/physionet.org/files/mimic-cxr/2.0.0/files/p10/p10975446/s58917552.txt", "r")
text = f.read()
f.close()
print(text)

                                 FINAL REPORT
 PORTABLE CHEST, ___
 
 COMPARISON:  ___ chest x-ray.
 
 FINDINGS:  Cardiac silhouette remains enlarged and is accompanied by
 persistent pulmonary vascular congestion and interstitial edema.  Patchy
 bibasilar atelectasis also appears similar compared to the prior study.



In [12]:
def clean_section(text):
    text = re.sub(r'[\S]+:', '', text)
    text = re.sub(r"_+", "_", text)  # Remove multiple underscores
    text = re.sub(r"\s\s+", " ", text)
    text = re.sub("[^a-zA-Z0-9 :.,-]", "", text)
    text = re.sub(r" +", " ", text)
    text = text.strip()
    return text

In [13]:
import string
def preprocess_report(text):
    # Remove unnecessary and insensible parts
    text = re.sub(r"EXAMINATION:.*", "", text)  # Remove EXAMINATION line
    text = re.sub(r"WET READ:.*", "", text)  # Remove WET READ line
    text = re.sub(r"FINAL REPORT", "", text)  # Remove FINAL REPORT line
    text = re.sub(r"STUDY:.*", "", text)  # Remove STUDY line
    text = re.sub(r"COMPARISON:.*", "", text)  # Remove COMPARISON section
    text = re.sub(r"TECHNIQUE:.*", "", text)  # Remove TECHNIQUE section
    text = re.sub(r"_+", "_", text)  # Remove multiple underscores

    # Clean up excessive newlines and spaces
    text = re.sub(r"\s\s+", " ", text)
    text = re.sub("[^a-zA-Z0-9 :.,-]", "", text)
    text = re.sub(r" +", " ", text)
    text = text.strip()
    return text

In [14]:
def extract_sections(report_text):
    findings_pattern = r'FINDINGS:[\s\S]*:'
    impression_pattern = r'IMPRESSION:[\s\S]*'

    findings_match = re.search(findings_pattern, report_text, re.IGNORECASE)
    impression_match = re.search(impression_pattern, report_text, re.IGNORECASE)

    findings = findings_match.group().strip() if findings_match else None
    impression = impression_match.group().strip() if impression_match else None

    if findings is None and impression is None:
        return preprocess_report(report_text)
    elif impression is None:
        return clean_section(findings)
    elif findings is None:
        return clean_section(impression)
    else:
        return clean_section(findings) + " " + clean_section(impression)

    return findings, impression

In [15]:
img_positions = {}

In [16]:
frontal = {"antero-posterior", "postero-anterior", ""}
lateral = {"lateral", "left lateral"}

In [17]:
with open("../data/mimic-cxr-2.0.0-metadata.csv") as f:
    lines = [line.rstrip() for line in f][1:]
    lines = [line for line in lines if line]
    
    for line in lines:
        fields = line.split(",")
        path = osp.join(f"p{fields[1][:2]}", f"p{fields[1]}", f"s{fields[2]}", f"{fields[0]}.jpg")
        if f"s{fields[2]}" not in img_positions:
            if fields[-2] in frontal:
                img_positions[f"s{fields[2]}"] = [(path, "frontal")]
            elif fields[-2] in lateral:
                img_positions[f"s{fields[2]}"] = [(path, "lateral")]
        else:
            if fields[-2] in frontal:
                img_positions[f"s{fields[2]}"].append((path, "frontal"))
            elif fields[-2] in lateral:
                img_positions[f"s{fields[2]}"].append((path, "lateral"))


In [18]:
list(img_positions.items())[:5]

[('s50414267',
  [('p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg',
    'frontal'),
   ('p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg',
    'lateral')]),
 ('s53189527',
  [('p10/p10000032/s53189527/2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab.jpg',
    'frontal'),
   ('p10/p10000032/s53189527/e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c.jpg',
    'lateral')]),
 ('s53911762',
  [('p10/p10000032/s53911762/68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714.jpg',
    'frontal'),
   ('p10/p10000032/s53911762/fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818.jpg',
    'frontal')]),
 ('s56699142',
  [('p10/p10000032/s56699142/ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c.jpg',
    'frontal')]),
 ('s57375967',
  [('p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg',
    'frontal'),
   ('p10/p10000764/s57375967/b79e55c3-735ce5ac-64412506-cdc9ea79-f1af521f.jpg',
    'lateral'),
   ('p10/p10000764/s57375967/dcfeeac4-1597e318-d0e673

In [19]:
lines[0].split(",")

['02aa804e-bde0afdd-112c0b34-7bc16630-4e384014',
 '10000032',
 '50414267',
 'CHEST (PA AND LAT)',
 'PA',
 '3056',
 '2544',
 '21800506',
 '213014.53100000002',
 'CHEST (PA AND LAT)',
 'postero-anterior',
 'Erect']

In [20]:
ALL_IMG_PATHS["train"][0]

'p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg'

In [21]:
extract_sections(text)

'PORTABLE CHEST, FINDINGS: Cardiac silhouette remains enlarged and is accompanied by persistent pulmonary vascular congestion and interstitial edema. Patchy bibasilar atelectasis also appears similar compared to the prior study.'

In [22]:
def get_report_from_path(path):
    try:
        f = open(path, "r")
        txt = extract_sections(f.read().strip())
        f.close()
    except FileNotFoundError as e:
        print(str(e))
        return None
    return txt

In [23]:
def images_n_locs_2_txt(study_id):
    study_id = study_id.split(".")[0]
    images = img_positions[study_id]

    strings = []
    for img_path_pairs in images:
        strings.append("[LOC_DELIM]".join(img_path_pairs))

    return "[VIEW_DELIM]".join(strings)

In [24]:
import csv

for sname, split in ALL_TXT_PATHS_PATIENT.items():
    with open(f"MIMIC_JPG_{sname[:5]}.tsv", 'w') as f:
        writer = csv.writer(f, delimiter='\t')

        for patient_id, studies in tqdm(split.items()):
            studies = list(studies.items())
            study_lim = len(studies) if len(studies) == 1 else len(studies) - 1
    
            if study_lim == 1:
                cur_study_id, cur_report_path = studies[0]
                cur_txt = get_report_from_path(cur_report_path)

                try:
                    cur_image_info = images_n_locs_2_txt(cur_study_id)
                except:
                    print("Invalid image position")
                    continue

                writer.writerow([cur_txt, cur_image_info])
            else:
                for study_idx in range(study_lim):
                    cur_study_id, cur_report_path = studies[study_idx]
                    next_study_id, next_report_path = studies[study_idx+1]
    
                    cur_txt = get_report_from_path(cur_report_path)
                    next_txt = get_report_from_path(next_report_path)
    
                    combined_txt = f"{cur_txt}[NEXT_TXT]{next_txt}"

                    try:
                        cur_image_info = images_n_locs_2_txt(cur_study_id)
                        next_image_info = images_n_locs_2_txt(next_study_id)
                    except:
                        print("Invalid image position")
                        continue

                    combined_img_info = f"{cur_image_info}[NEXT_IMG]{next_image_info}"
                    writer.writerow([combined_txt, combined_img_info])


  0%|          | 33/64213 [00:00<22:33, 47.41it/s]

Invalid image position
Invalid image position


  0%|          | 120/64213 [00:02<16:43, 63.85it/s]

Invalid image position


  0%|          | 237/64213 [00:04<16:28, 64.69it/s]

...

 75%|███████▍  | 219/293 [00:05<00:01, 48.32it/s]

Invalid image position
Invalid image position


 88%|████████▊ | 259/293 [00:06<00:00, 53.01it/s]

Invalid image position
Invalid image position


100%|██████████| 293/293 [00:07<00:00, 40.58it/s]


In [25]:
l = []

In [26]:
with open("MIMIC_JPG_train_trimmed.tsv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for item in reader:
        if "[NEXT_IMG]" not in item[-1]:
            cur_img = item[-1]
            cur_img_count, next_img_count = cur_img.count("LOC_DELIM"), 0
        else:
            cur_img, next_img = item[-1].split("[NEXT_IMG]")
            cur_img_count, next_img_count = cur_img.count("LOC_DELIM"), next_img.count("LOC_DELIM")
        l.append((cur_img_count, next_img_count))


FileNotFoundError: [Errno 2] No such file or directory: 'MIMIC_JPG_train_trimmed.tsv'

In [29]:
LIMIT = 2

with open("MIMIC_JPG_test.tsv", "r") as f, open("MIMIC_JPG_test_trimmed.tsv", "w") as f_write:
    reader = csv.reader(f, delimiter="\t")
    writer = csv.writer(f_write, delimiter="\t")
    for item in tqdm(reader):
        if "[NEXT_IMG]" not in item[-1]:
            cur_img = item[-1]
            cur_img_count, next_img_count = cur_img.count("LOC_DELIM"), 0
        else:
            cur_img, next_img = item[-1].split("[NEXT_IMG]")
            cur_img_count, next_img_count = cur_img.count("LOC_DELIM"), next_img.count("LOC_DELIM")

        if cur_img_count > LIMIT:
            cur_img = "[VIEW_DELIM]".join(cur_img.split("[VIEW_DELIM]")[:LIMIT])
        if next_img_count > LIMIT:
            next_img = "[VIEW_DELIM]".join(next_img.split("[VIEW_DELIM]")[:LIMIT])

        views = cur_img.split("[VIEW_DELIM]")
        views = sorted(views, key=lambda view: view.split("[LOC_DELIM]")[-1])
        cur_img = "[VIEW_DELIM]".join(views)

        if next_img_count != 0:
            views = next_img.split("[VIEW_DELIM]")
            views = sorted(views, key=lambda view: view.split("[LOC_DELIM]")[-1])
            next_img = "[VIEW_DELIM]".join(views)

        if "[NEXT_IMG]" in item[-1]:
            cur_img = f"{cur_img}[NEXT_IMG]{next_img}"

        item[-1] = cur_img
        if "[NEXT_IMG]" not in item[-1]:
            cur_img = item[-1]
            cur_img_count, next_img_count = cur_img.count("LOC_DELIM"), 0
        else:
            cur_img, next_img = item[-1].split("[NEXT_IMG]")
            cur_img_count, next_img_count = cur_img.count("LOC_DELIM"), next_img.count("LOC_DELIM")

        item = (*item, cur_img_count, next_img_count)

        writer.writerow(item)

2934it [00:00, 14064.95it/s]


In [ ]:
from collections import Counter
c = Counter(l)

In [75]:
c

Counter({(1, 1): 52654,
         (2, 2): 40642,
         (2, 0): 34681,
         (1, 2): 25935,
         (2, 1): 25884,
         (1, 0): 8285})

In [61]:
c

Counter({(1, 1): 52654,
         (2, 0): 28731,
         (2, 2): 28532,
         (1, 2): 21655,
         (2, 1): 21549,
         (1, 0): 8285,
         (3, 0): 5950,
         (2, 3): 4962,
         (3, 2): 4950,
         (3, 1): 4335,
         (1, 3): 4280,
         (3, 3): 2198})

In [47]:
c

Counter({(1, 1): 52654,
         (2, 0): 28731,
         (2, 2): 28534,
         (1, 2): 21655,
         (2, 1): 21550,
         (1, 0): 8285,
         (3, 0): 5378,
         (2, 3): 4514,
         (3, 2): 4501,
         (3, 1): 3869,
         (1, 3): 3794,
         (3, 3): 1646,
         (4, 0): 544,
         (1, 4): 439,
         (4, 1): 429,
         (4, 2): 422,
         (2, 4): 421,
         (4, 3): 240,
         (3, 4): 222,
         (4, 4): 59,
         (1, 5): 36,
         (5, 1): 30,
         (5, 2): 22,
         (5, 0): 20,
         (2, 5): 17,
         (2, 6): 9,
         (5, 3): 9,
         (6, 0): 8,
         (1, 6): 6,
         (3, 5): 6,
         (6, 1): 5,
         (4, 5): 4,
         (6, 6): 3,
         (6, 4): 3,
         (1, 8): 3,
         (6, 2): 3,
         (1, 7): 2,
         (4, 6): 2,
         (5, 4): 2,
         (7, 6): 1,
         (8, 4): 1,
         (9, 1): 1,
         (8, 1): 1,
         (8, 2): 1,
         (2, 11): 1,
         (11, 2): 1})